In [1]:
import org.apache.spark.sql._
import scala.sys.process._

import org.apache.spark.sql._

import scala.sys.process._

In [2]:
// So now we need to configure Spark to use Iceberg
// See https://iceberg.apache.org/docs/1.6.0/spark-configuration/ & https://iceberg.apache.org/docs/1.6.0/spark-getting-started/
// We'll use the "hadoop" (aka file) catalog & /high-performance-spark-examples/warehouse for the location
val spark = (SparkSession.builder.master("local")
             // Setup the extensions
             .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
             .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
             .config("spark.sql.catalog.local.type", "hadoop")
             .config("spark.sql.catalog.local.warehouse", "/high-performance-spark-examples/warehouse")
             .getOrCreate()
             )
import spark._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/08/13 22:49:02 INFO SparkContext: Running Spark version 3.5.2
24/08/13 22:49:02 INFO SparkContext: OS info Linux, 6.5.0-17-generic, amd64
24/08/13 22:49:02 INFO SparkContext: Java version 11.0.24
24/08/13 22:49:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/13 22:49:02 INFO ResourceUtils: ==============================================================
24/08/13 22:49:02 INFO ResourceUtils: No custom resources configured for spark.driver.
24/08/13 22:49:02 INFO ResourceUtils: ==============================================================
24/08/13 22:49:02 INFO SparkContext: Submitted application: de374f24-5470-49f4-91e1-2f627c291cc4
24/08/13 22:49:02 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, scr

spark: SparkSession = org.apache.spark.sql.SparkSession@133f6af3
import spark._

In [3]:
// Load the current data
val df = spark.read.option("header", "true").option("inferSchema", "true").csv("/high-performance-spark-examples/data/fetched/2021")

24/08/13 22:49:03 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/08/13 22:49:03 INFO SharedState: Warehouse path is 'file:/high-performance-spark-examples/iceberg-workshop/spark-warehouse'.
24/08/13 22:49:04 INFO InMemoryFileIndex: It took 24 ms to list leaf files for 1 paths.
24/08/13 22:49:04 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/08/13 22:49:05 INFO FileSourceStrategy: Pushed Filters: 
24/08/13 22:49:05 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
24/08/13 22:49:05 INFO CodeGenerator: Code generated in 136.990334 ms
24/08/13 22:49:05 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 199.5 KiB, free 13.8 GiB)
24/08/13 22:49:05 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 13.8 GiB)
24/08/13 22:49:05 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on fa3024105736:40255

df: DataFrame = [EmployerName: string, EmployerId: string ... 25 more fields]

In [4]:
// Drop existing table if present & create new table
spark.sql("DROP TABLE IF EXISTS local.uk_gender_pay_data")

24/08/13 22:49:07 INFO CatalogUtil: Loading custom FileIO implementation: org.apache.iceberg.hadoop.HadoopFileIO
24/08/13 22:49:07 INFO BaseMetastoreCatalog: Table loaded by catalog: local.uk_gender_pay_data


res3: DataFrame = []

In [5]:
// Write the data out
df.write.saveAsTable("local.uk_gender_pay_data")

24/08/13 22:49:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/08/13 22:49:07 INFO BaseMetastoreCatalog: Table properties set at catalog level through catalog properties: {}
24/08/13 22:49:07 INFO BaseMetastoreCatalog: Table properties enforced at catalog level through catalog properties: {}
24/08/13 22:49:07 INFO SparkWrite: Requesting 0 bytes advisory partition size for table uk_gender_pay_data
24/08/13 22:49:07 INFO SparkWrite: Requesting UnspecifiedDistribution as write distribution for table uk_gender_pay_data
24/08/13 22:49:07 INFO SparkWrite: Requesting [] as write ordering for table uk_gender_pay_data
24/08/13 22:49:07 INFO FileSourceStrategy: Pushed Filters: 
24/08/13 22:49:07 INFO FileSourceStrategy: Post-Scan Filters: 
24/08/13 22:49:07 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 199.4 KiB, free 13.8 GiB)
24/08/13

In [6]:
"ls /high-performance-spark-examples/warehouse/uk_gender_pay_data/metadata/".!!

res5: String = """c8c28535-2d82-40b1-a36d-fcfc8885f1be-m0.avro
snap-3775120664731591-1-c8c28535-2d82-40b1-a36d-fcfc8885f1be.avro
v1.metadata.json
version-hint.text
"""

In [7]:
"cat  /high-performance-spark-examples/warehouse/uk_gender_pay_data/metadata/v1.metadata.json".!!

res6: String = """{
  "format-version" : 2,
  "table-uuid" : "f464c7c6-9962-453c-9705-aa243b3e7f55",
  "location" : "/high-performance-spark-examples/warehouse/uk_gender_pay_data",
  "last-sequence-number" : 1,
  "last-updated-ms" : 1723589348730,
  "last-column-id" : 27,
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "EmployerName",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "EmployerId",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 3,
      "name" : "Address",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 4,
      "name" : "PostCode",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 5,
      "name" : "CompanyNumber",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 6,
      "name" : "SicCodes",
...

In [8]:
// Java SDK time imports
import java.util.HashMap
import java.util.Map

import org.apache.iceberg.Table
import org.apache.iceberg.catalog.TableIdentifier
import org.apache.iceberg.hadoop.HadoopCatalog

// And to handle java types
import scala.jdk.CollectionConverters._

import java.util.HashMap

import java.util.Map


import org.apache.iceberg.Table

import org.apache.iceberg.catalog.TableIdentifier

import org.apache.iceberg.hadoop.HadoopCatalog

// And to handle java types

import scala.jdk.CollectionConverters._

In [9]:
val catalog = new HadoopCatalog(spark.sparkContext.hadoopConfiguration, "/high-performance-spark-examples/warehouse")

24/08/13 22:49:09 INFO CatalogUtil: Loading custom FileIO implementation: org.apache.iceberg.hadoop.HadoopFileIO


catalog: HadoopCatalog = HadoopCatalog{name=hadoop, location=/high-performance-spark-examples/warehouse}

In [10]:
val name = TableIdentifier.of("uk_gender_pay_data")

name: TableIdentifier = uk_gender_pay_data

In [11]:
val table = catalog.loadTable(name)

24/08/13 22:49:09 INFO BaseMetastoreCatalog: Table loaded by catalog: hadoop.uk_gender_pay_data


table: Table = hadoop.uk_gender_pay_data

In [12]:
val snapshots = table.snapshots().asScala.toList
snapshots

snapshots: List[org.apache.iceberg.Snapshot] = List(
  BaseSnapshot{id=3775120664731591, timestamp_ms=1723589348730, operation=append, summary={spark.app.id=local-1723589343398, added-data-files=1, added-records=15395, added-files-size=1294005, changed-partition-count=1, total-records=15395, total-files-size=1294005, total-data-files=1, total-delete-files=0, total-position-deletes=0, total-equality-deletes=0, engine-version=3.5.2, app-id=local-1723589343398, engine-name=spark, iceberg-version=Apache Iceberg 1.6.0 (commit 229d8f6fcd109e6c8943ea7cbb41dab746c6d0ed)}, manifest-list=/high-performance-spark-examples/warehouse/uk_gender_pay_data/metadata/snap-3775120664731591-1-c8c28535-2d82-40b1-a36d-fcfc8885f1be.avro, schema-id=0}
)
res11_1: List[org.apache.iceberg.Snapshot] = List(
  BaseSnapshot{id=3775120664731591, timestamp_ms=1723589348730, operation=append, summary={spark.app.id=local-1723589343398, added-data-files=1, added-records=15395, added-files-size=1294005, changed-partition-c

In [13]:
val snapshot = snapshots(0).snapshotId()

snapshot: Long = 3775120664731591L

In [16]:
val altSnapshotQuery = spark.sql("SELECT * FROM local.uk_gender_pay_data.snapshots")
altSnapshotQuery.show()

24/08/13 22:51:31 INFO BaseMetastoreCatalog: Table loaded by catalog: local.uk_gender_pay_data
24/08/13 22:51:31 INFO V2ScanRelationPushDown: 
Output: committed_at#377, snapshot_id#378L, parent_id#379L, operation#380, manifest_list#381, summary#382
         
24/08/13 22:51:31 INFO SparkPartitioningAwareScan: Reporting UnknownPartitioning with 1 partition(s) for table local.uk_gender_pay_data.snapshots
24/08/13 22:51:31 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:51:31 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 29.2 KiB, free 13.8 GiB)
24/08/13 22:51:31 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on fa3024105736:40255 (size: 29.2 KiB, free: 13.8 GiB)
24/08/13 22:51:31 INFO SparkContext: Created broadcast 13 from broadcast at SparkBatch.java:79
24/08/13 22:51:31 INFO MemoryStore: Block broadcast_14 stored as values in memory (estimated size 32.0 KiB, free 13.

+--------------------+----------------+---------+---------+--------------------+--------------------+
|        committed_at|     snapshot_id|parent_id|operation|       manifest_list|             summary|
+--------------------+----------------+---------+---------+--------------------+--------------------+
|2024-08-13 22:49:...|3775120664731591|     NULL|   append|/high-performance...|{spark.app.id -> ...|
+--------------------+----------------+---------+---------+--------------------+--------------------+



altSnapshotQuery: DataFrame = [committed_at: timestamp, snapshot_id: bigint ... 4 more fields]

In [17]:
val altSnapshotId = spark.sql("SELECT snapshot_id FROM local.uk_gender_pay_data.snapshots").collect()(0)

24/08/13 22:52:01 INFO BaseMetastoreCatalog: Table loaded by catalog: local.uk_gender_pay_data
24/08/13 22:52:01 INFO V2ScanRelationPushDown: 
Output: snapshot_id#426L
         
24/08/13 22:52:01 INFO SparkPartitioningAwareScan: Reporting UnknownPartitioning with 1 partition(s) for table local.uk_gender_pay_data.snapshots
24/08/13 22:52:01 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:52:02 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 29.2 KiB, free 13.8 GiB)
24/08/13 22:52:02 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on fa3024105736:40255 (size: 29.2 KiB, free: 13.8 GiB)
24/08/13 22:52:02 INFO SparkContext: Created broadcast 16 from broadcast at SparkBatch.java:79
24/08/13 22:52:02 INFO MemoryStore: Block broadcast_17 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:52:02 INFO MemoryStore: Block broadcast_17_piece0 stored as by

altSnapshotId: Row = [3775120664731591]

In [18]:
spark.sql("SELECT * FROM local.uk_gender_pay_data WHERE isnull(responsibleperson) LIMIT 5").show()

24/08/13 22:52:13 INFO V2ScanRelationPushDown: 
Pushing operators to local.uk_gender_pay_data
Pushed Filters: ResponsiblePerson IS NULL
Post-Scan Filters: isnull(ResponsiblePerson#461)
         
24/08/13 22:52:13 INFO V2ScanRelationPushDown: 
Output: EmployerName#440, EmployerId#441, Address#442, PostCode#443, CompanyNumber#444, SicCodes#445, DiffMeanHourlyPercent#446, DiffMedianHourlyPercent#447, DiffMeanBonusPercent#448, DiffMedianBonusPercent#449, MaleBonusPercent#450, FemaleBonusPercent#451, MaleLowerQuartile#452, FemaleLowerQuartile#453, MaleLowerMiddleQuartile#454, FemaleLowerMiddleQuartile#455, MaleUpperMiddleQuartile#456, FemaleUpperMiddleQuartile#457, MaleTopQuartile#458, FemaleTopQuartile#459, CompanyLinkToGPGInfo#460, ResponsiblePerson#461, EmployerSize#462, CurrentName#463, SubmittedAfterTheDeadline#464, DueDate#465, DateSubmitted#466
         
24/08/13 22:52:13 INFO SnapshotScan: Scanning table local.uk_gender_pay_data snapshot 3775120664731591 created at 2024-08-13T22:49:

+--------------------+----------+--------------------+--------------------+-------------+--------+---------------------+-----------------------+--------------------+----------------------+----------------+------------------+-----------------+-------------------+-----------------------+-------------------------+-----------------------+-------------------------+---------------+-----------------+--------------------+-----------------+--------------------+-----------+-------------------------+---------+-------------+
|        EmployerName|EmployerId|             Address|            PostCode|CompanyNumber|SicCodes|DiffMeanHourlyPercent|DiffMedianHourlyPercent|DiffMeanBonusPercent|DiffMedianBonusPercent|MaleBonusPercent|FemaleBonusPercent|MaleLowerQuartile|FemaleLowerQuartile|MaleLowerMiddleQuartile|FemaleLowerMiddleQuartile|MaleUpperMiddleQuartile|FemaleUpperMiddleQuartile|MaleTopQuartile|FemaleTopQuartile|CompanyLinkToGPGInfo|ResponsiblePerson|        EmployerSize|CurrentName|SubmittedAfte

In [19]:
// We can also list snapshots with the select
spark.sql("SELECT * FROM local.uk_gender_pay_data.snapshots").show()

24/08/13 22:52:14 INFO V2ScanRelationPushDown: 
Output: committed_at#635, snapshot_id#636L, parent_id#637L, operation#638, manifest_list#639, summary#640
         
24/08/13 22:52:14 INFO SparkPartitioningAwareScan: Reporting UnknownPartitioning with 1 partition(s) for table local.uk_gender_pay_data.snapshots
24/08/13 22:52:14 INFO MemoryStore: Block broadcast_22 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:52:14 INFO MemoryStore: Block broadcast_22_piece0 stored as bytes in memory (estimated size 29.2 KiB, free 13.8 GiB)
24/08/13 22:52:14 INFO BlockManagerInfo: Added broadcast_22_piece0 in memory on fa3024105736:40255 (size: 29.2 KiB, free: 13.8 GiB)
24/08/13 22:52:14 INFO SparkContext: Created broadcast 22 from broadcast at SparkBatch.java:79
24/08/13 22:52:14 INFO MemoryStore: Block broadcast_23 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:52:14 INFO MemoryStore: Block broadcast_23_piece0 stored as bytes in memory 

+--------------------+----------------+---------+---------+--------------------+--------------------+
|        committed_at|     snapshot_id|parent_id|operation|       manifest_list|             summary|
+--------------------+----------------+---------+---------+--------------------+--------------------+
|2024-08-13 22:49:...|3775120664731591|     NULL|   append|/high-performance...|{spark.app.id -> ...|
+--------------------+----------------+---------+---------+--------------------+--------------------+



In [20]:
// And the files!
// We can also list snapshots with the select
spark.sql("SELECT * FROM local.uk_gender_pay_data.files").show()

24/08/13 22:52:15 INFO V2ScanRelationPushDown: 
Output: content#683, file_path#684, file_format#685, spec_id#686, record_count#687L, file_size_in_bytes#688L, column_sizes#689, value_counts#690, null_value_counts#691, nan_value_counts#692, lower_bounds#693, upper_bounds#694, key_metadata#695, split_offsets#696, equality_ids#697, sort_order_id#698, readable_metrics#699
         
24/08/13 22:52:15 INFO SnapshotScan: Scanning table local.uk_gender_pay_data snapshot 3775120664731591 created at 2024-08-13T22:49:08.730+00:00 with filter true
24/08/13 22:52:15 INFO SparkPartitioningAwareScan: Reporting UnknownPartitioning with 1 partition(s) for table local.uk_gender_pay_data.files
24/08/13 22:52:15 INFO MemoryStore: Block broadcast_25 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:52:15 INFO MemoryStore: Block broadcast_25_piece0 stored as bytes in memory (estimated size 32.4 KiB, free 13.8 GiB)
24/08/13 22:52:15 INFO BlockManagerInfo: Added broadcast_25_piece

+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|content|           file_path|file_format|spec_id|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|    nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|    readable_metrics|
+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|      0|/high-performance...|    PARQUET|      0|       15395|           1294005|{1 -> 139005, 2 -...|{1 -> 15395, 2 ->...|{1 -> 0, 2 -> 157...|{8 

In [21]:
// Lets take a quick look and see
spark.sql("SELECT * FROM local.uk_gender_pay_data WHERE isnull(responsibleperson) LIMIT 5").show()

24/08/13 22:52:16 INFO V2ScanRelationPushDown: 
Pushing operators to local.uk_gender_pay_data
Pushed Filters: ResponsiblePerson IS NULL
Post-Scan Filters: isnull(ResponsiblePerson#829)
         
24/08/13 22:52:16 INFO V2ScanRelationPushDown: 
Output: EmployerName#808, EmployerId#809, Address#810, PostCode#811, CompanyNumber#812, SicCodes#813, DiffMeanHourlyPercent#814, DiffMedianHourlyPercent#815, DiffMeanBonusPercent#816, DiffMedianBonusPercent#817, MaleBonusPercent#818, FemaleBonusPercent#819, MaleLowerQuartile#820, FemaleLowerQuartile#821, MaleLowerMiddleQuartile#822, FemaleLowerMiddleQuartile#823, MaleUpperMiddleQuartile#824, FemaleUpperMiddleQuartile#825, MaleTopQuartile#826, FemaleTopQuartile#827, CompanyLinkToGPGInfo#828, ResponsiblePerson#829, EmployerSize#830, CurrentName#831, SubmittedAfterTheDeadline#832, DueDate#833, DateSubmitted#834
         
24/08/13 22:52:16 INFO SnapshotScan: Scanning table local.uk_gender_pay_data snapshot 3775120664731591 created at 2024-08-13T22:49:

+--------------------+----------+--------------------+--------------------+-------------+--------+---------------------+-----------------------+--------------------+----------------------+----------------+------------------+-----------------+-------------------+-----------------------+-------------------------+-----------------------+-------------------------+---------------+-----------------+--------------------+-----------------+--------------------+-----------+-------------------------+---------+-------------+
|        EmployerName|EmployerId|             Address|            PostCode|CompanyNumber|SicCodes|DiffMeanHourlyPercent|DiffMedianHourlyPercent|DiffMeanBonusPercent|DiffMedianBonusPercent|MaleBonusPercent|FemaleBonusPercent|MaleLowerQuartile|FemaleLowerQuartile|MaleLowerMiddleQuartile|FemaleLowerMiddleQuartile|MaleUpperMiddleQuartile|FemaleUpperMiddleQuartile|MaleTopQuartile|FemaleTopQuartile|CompanyLinkToGPGInfo|ResponsiblePerson|        EmployerSize|CurrentName|SubmittedAfte

In [22]:
spark.sql("DELETE FROM local.uk_gender_pay_data WHERE isnull(responsibleperson)")

24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_7_piece0 on fa3024105736:40255 in memory (size: 29.4 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_8_piece0 on fa3024105736:40255 in memory (size: 29.3 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_13_piece0 on fa3024105736:40255 in memory (size: 29.2 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_23_piece0 on fa3024105736:40255 in memory (size: 29.2 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_6_piece0 on fa3024105736:40255 in memory (size: 8.5 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_16_piece0 on fa3024105736:40255 in memory (size: 29.2 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_18_piece0 on fa3024105736:40255 in memory (size: 4.9 KiB, free: 13.8 GiB)
24/08/13 22:52:16 INFO BlockManagerInfo: Removed broadcast_15_piece

res21: DataFrame = []

In [23]:
// Make sure the data is gone
spark.sql("SELECT * FROM local.uk_gender_pay_data WHERE isnull(responsibleperson) LIMIT 5").show()

24/08/13 22:52:18 INFO V2ScanRelationPushDown: 
Pushing operators to local.uk_gender_pay_data
Pushed Filters: ResponsiblePerson IS NULL
Post-Scan Filters: isnull(ResponsiblePerson#1146)
         
24/08/13 22:52:18 INFO V2ScanRelationPushDown: 
Output: EmployerName#1125, EmployerId#1126, Address#1127, PostCode#1128, CompanyNumber#1129, SicCodes#1130, DiffMeanHourlyPercent#1131, DiffMedianHourlyPercent#1132, DiffMeanBonusPercent#1133, DiffMedianBonusPercent#1134, MaleBonusPercent#1135, FemaleBonusPercent#1136, MaleLowerQuartile#1137, FemaleLowerQuartile#1138, MaleLowerMiddleQuartile#1139, FemaleLowerMiddleQuartile#1140, MaleUpperMiddleQuartile#1141, FemaleUpperMiddleQuartile#1142, MaleTopQuartile#1143, FemaleTopQuartile#1144, CompanyLinkToGPGInfo#1145, ResponsiblePerson#1146, EmployerSize#1147, CurrentName#1148, SubmittedAfterTheDeadline#1149, DueDate#1150, DateSubmitted#1151
         
24/08/13 22:52:18 INFO SnapshotScan: Scanning table local.uk_gender_pay_data snapshot 43324083639653821

+------------+----------+-------+--------+-------------+--------+---------------------+-----------------------+--------------------+----------------------+----------------+------------------+-----------------+-------------------+-----------------------+-------------------------+-----------------------+-------------------------+---------------+-----------------+--------------------+-----------------+------------+-----------+-------------------------+-------+-------------+
|EmployerName|EmployerId|Address|PostCode|CompanyNumber|SicCodes|DiffMeanHourlyPercent|DiffMedianHourlyPercent|DiffMeanBonusPercent|DiffMedianBonusPercent|MaleBonusPercent|FemaleBonusPercent|MaleLowerQuartile|FemaleLowerQuartile|MaleLowerMiddleQuartile|FemaleLowerMiddleQuartile|MaleUpperMiddleQuartile|FemaleUpperMiddleQuartile|MaleTopQuartile|FemaleTopQuartile|CompanyLinkToGPGInfo|ResponsiblePerson|EmployerSize|CurrentName|SubmittedAfterTheDeadline|DueDate|DateSubmitted|
+------------+----------+-------+--------+------

In [24]:
// Yay! ok now lets travel back in time
spark.sql(f"SELECT * FROM local.uk_gender_pay_data VERSION AS OF ${snapshot} WHERE isnull(responsibleperson) LIMIT 5").show()

24/08/13 22:52:18 INFO V2ScanRelationPushDown: 
Pushing operators to local.uk_gender_pay_data
Pushed Filters: ResponsiblePerson IS NULL
Post-Scan Filters: isnull(ResponsiblePerson#1341)
         
24/08/13 22:52:18 INFO V2ScanRelationPushDown: 
Output: EmployerName#1320, EmployerId#1321, Address#1322, PostCode#1323, CompanyNumber#1324, SicCodes#1325, DiffMeanHourlyPercent#1326, DiffMedianHourlyPercent#1327, DiffMeanBonusPercent#1328, DiffMedianBonusPercent#1329, MaleBonusPercent#1330, FemaleBonusPercent#1331, MaleLowerQuartile#1332, FemaleLowerQuartile#1333, MaleLowerMiddleQuartile#1334, FemaleLowerMiddleQuartile#1335, MaleUpperMiddleQuartile#1336, FemaleUpperMiddleQuartile#1337, MaleTopQuartile#1338, FemaleTopQuartile#1339, CompanyLinkToGPGInfo#1340, ResponsiblePerson#1341, EmployerSize#1342, CurrentName#1343, SubmittedAfterTheDeadline#1344, DueDate#1345, DateSubmitted#1346
         
24/08/13 22:52:18 INFO SnapshotScan: Scanning table local.uk_gender_pay_data snapshot 3775120664731591 

+--------------------+----------+--------------------+--------------------+-------------+--------+---------------------+-----------------------+--------------------+----------------------+----------------+------------------+-----------------+-------------------+-----------------------+-------------------------+-----------------------+-------------------------+---------------+-----------------+--------------------+-----------------+--------------------+-----------+-------------------------+---------+-------------+
|        EmployerName|EmployerId|             Address|            PostCode|CompanyNumber|SicCodes|DiffMeanHourlyPercent|DiffMedianHourlyPercent|DiffMeanBonusPercent|DiffMedianBonusPercent|MaleBonusPercent|FemaleBonusPercent|MaleLowerQuartile|FemaleLowerQuartile|MaleLowerMiddleQuartile|FemaleLowerMiddleQuartile|MaleUpperMiddleQuartile|FemaleUpperMiddleQuartile|MaleTopQuartile|FemaleTopQuartile|CompanyLinkToGPGInfo|ResponsiblePerson|        EmployerSize|CurrentName|SubmittedAfte

In [25]:
// Or the same with option + DF syntax
spark.read.option("snapshot-id", f"${snapshot}").table("local.uk_gender_pay_data").show()

24/08/13 22:52:18 INFO V2ScanRelationPushDown: 
Output: EmployerName#1515, EmployerId#1516, Address#1517, PostCode#1518, CompanyNumber#1519, SicCodes#1520, DiffMeanHourlyPercent#1521, DiffMedianHourlyPercent#1522, DiffMeanBonusPercent#1523, DiffMedianBonusPercent#1524, MaleBonusPercent#1525, FemaleBonusPercent#1526, MaleLowerQuartile#1527, FemaleLowerQuartile#1528, MaleLowerMiddleQuartile#1529, FemaleLowerMiddleQuartile#1530, MaleUpperMiddleQuartile#1531, FemaleUpperMiddleQuartile#1532, MaleTopQuartile#1533, FemaleTopQuartile#1534, CompanyLinkToGPGInfo#1535, ResponsiblePerson#1536, EmployerSize#1537, CurrentName#1538, SubmittedAfterTheDeadline#1539, DueDate#1540, DateSubmitted#1541
         
24/08/13 22:52:18 INFO SnapshotScan: Scanning table local.uk_gender_pay_data snapshot 3775120664731591 created at 2024-08-13T22:49:08.730+00:00 with filter true
24/08/13 22:52:18 INFO BaseDistributedDataScan: Planning file tasks locally for table local.uk_gender_pay_data
24/08/13 22:52:18 INFO Spar

+--------------------+----------+--------------------+--------------------+-------------+--------+---------------------+-----------------------+--------------------+----------------------+----------------+------------------+-----------------+-------------------+-----------------------+-------------------------+-----------------------+-------------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-------------------------+-------------------+-------------------+
|        EmployerName|EmployerId|             Address|            PostCode|CompanyNumber|SicCodes|DiffMeanHourlyPercent|DiffMedianHourlyPercent|DiffMeanBonusPercent|DiffMedianBonusPercent|MaleBonusPercent|FemaleBonusPercent|MaleLowerQuartile|FemaleLowerQuartile|MaleLowerMiddleQuartile|FemaleLowerMiddleQuartile|MaleUpperMiddleQuartile|FemaleUpperMiddleQuartile|     MaleTopQuartile|FemaleTopQuartile|CompanyLinkToGPGInfo|   ResponsiblePerson|     

In [28]:
spark.sql("DROP TABLE IF EXISTS local.uk_gender_pay_data_postcode")

res27: DataFrame = []

In [29]:
// Write the data out
df.registerTempTable("temp_table")
spark.sql("CREATE TABLE local.uk_gender_pay_data_postcode USING iceberg PARTITIONED BY (truncate(1, PostCode)) AS select * from temp_table")

24/08/13 22:55:33 INFO BaseMetastoreCatalog: Table properties set at catalog level through catalog properties: {}
24/08/13 22:55:33 INFO BaseMetastoreCatalog: Table properties enforced at catalog level through catalog properties: {}
24/08/13 22:55:33 INFO SparkWrite: Requesting 402653184 bytes advisory partition size for table uk_gender_pay_data_postcode
24/08/13 22:55:33 INFO SparkWrite: Requesting ClusteredDistribution(truncate(1, PostCode)) as write distribution for table uk_gender_pay_data_postcode
24/08/13 22:55:33 INFO SparkWrite: Requesting [] as write ordering for table uk_gender_pay_data_postcode
24/08/13 22:55:33 INFO FileSourceStrategy: Pushed Filters: 
24/08/13 22:55:33 INFO FileSourceStrategy: Post-Scan Filters: 
24/08/13 22:55:33 INFO MemoryStore: Block broadcast_50 stored as values in memory (estimated size 199.4 KiB, free 13.8 GiB)
24/08/13 22:55:33 INFO MemoryStore: Block broadcast_50_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 13.8 GiB)
24/08/13 22

res28_1: DataFrame = []

In [30]:
spark.sql("SELECT * FROM local.uk_gender_pay_data_postcode.files").show()

24/08/13 22:57:09 INFO BaseMetastoreCatalog: Table loaded by catalog: local.uk_gender_pay_data_postcode
24/08/13 22:57:09 INFO V2ScanRelationPushDown: 
Output: content#1818, file_path#1819, file_format#1820, spec_id#1821, partition#1822, record_count#1823L, file_size_in_bytes#1824L, column_sizes#1825, value_counts#1826, null_value_counts#1827, nan_value_counts#1828, lower_bounds#1829, upper_bounds#1830, key_metadata#1831, split_offsets#1832, equality_ids#1833, sort_order_id#1834, readable_metrics#1835
         
24/08/13 22:57:09 INFO SnapshotScan: Scanning table local.uk_gender_pay_data_postcode snapshot 446658414211481299 created at 2024-08-13T22:55:34.817+00:00 with filter true
24/08/13 22:57:09 INFO SparkPartitioningAwareScan: Reporting UnknownPartitioning with 1 partition(s) for table local.uk_gender_pay_data_postcode.files
24/08/13 22:57:09 INFO MemoryStore: Block broadcast_54 stored as values in memory (estimated size 32.0 KiB, free 13.8 GiB)
24/08/13 22:57:09 INFO MemoryStore: B

+-------+--------------------+-----------+-------+---------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|content|           file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|    nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|    readable_metrics|
+-------+--------------------+-----------+-------+---------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|      0|/high-performance...|    PARQUET|      0|      {9}|          57|             15272|{1 -> 282, 2 -> 2...|{1 ->

In [30]:
// Huzzah!

In [31]:
// Lets go take a look at a quick side-by-side test
"/high-performance-spark-examples/spark-upgrade/e2e_demo/scala/run_demo.sh".!!

+ python python_check.py
/high-performance-spark-examples/spark-upgrade/e2e_demo/scala/run_demo.sh: line 7: python: command not found


: 